In [2]:
%pip install selenium
%pip install beautifulsoup4

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [4]:
from selenium import webdriver
import csv
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium import webdriver
from bs4 import BeautifulSoup as bs
import re as re
import time
import pandas as pd

# User's first and last name
first_name = input("Enter the user's first name: ")
last_name = input("Enter the user's last name: ")
search_query = first_name+' '+last_name
# Initialize the WebDrive
driver_path = 'chromedriver-win64/chromedriver.exe'
user = input("Enter the username: ")
passwd = input("Enter the password: ")

# Set up the Chrome web driver
driver = webdriver.Chrome(driver_path)

# Navigate to the LinkedIn login page
driver.get('https://www.linkedin.com/login')

# Wait for the page to load
time.sleep(2)

# Fill in the login form
username = driver.find_element_by_id('username')
username.send_keys(user)
password = driver.find_element_by_id('password')
password.send_keys(passwd)
password.submit()

# Wait for the home page to load
time.sleep(2)

In [14]:
driver.get(f"https://www.linkedin.com/search/results/people/?keywords={search_query}")

# Extract data from the first 10 search results

profile_urls = []
results = driver.find_elements_by_class_name('search-results-container')
profiles = results[0].find_elements_by_class_name('reusable-search__result-container')[:10]
for profile in profiles:
    profile_urls.append(profile.find_element_by_css_selector('a.app-aware-link').get_attribute('href'))
profile_urls

['https://www.linkedin.com/in/diya-shah-48721626a?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAAEHiNCcBHSKq1ynvzizYCidNygahYIVSh18',
 'https://www.linkedin.com/in/diya-shah-b24832226?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAADi4RswBTIffEWizgfqmhE3ECYyY_N2e4uQ',
 'https://www.linkedin.com/in/diya-shah-bba073214?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAADYdUf4B8U0rOTV116TaAffX4LS5nC1YXmA',
 'https://www.linkedin.com/in/diya-shah?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAACzPZpUB_OC754FdasuqlVXaUbq9JR1lKjM',
 'https://www.linkedin.com/in/diya-shah-977260188?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAACwtA2sB4pBhTdXI4twso0oeZysomUwOQiQ',
 'https://www.linkedin.com/in/diya3333?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAADcmPWABwbmHNzUYEtRHTA13d9BvDVX1iJc',
 'https://www.linkedin.com/in/shah-diya?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAAEHDGBABqmo8ZVi0Zm2OouA5i2E-1ogPGvA',
 'https://www.linkedin.com/in/diya-shah-836375230?miniProfileUrn=urn%3Ali%3Afs_miniPro

In [15]:
driver.get(profile_urls[0])